In [17]:
# current submission is .79
# Things I'd like to do:
    # try unsupervised ML?
    # Try a NN or reinforcement or something?
    # Learn how to plot data and make charts?
    # Can we use the cabin column in a meaningful way for rows that have that?
    # find other ways to handle NaN Age?
    
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OneHotEncoder, minmax_scale
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
#!kaggle competitions download -c titanic
#!kaggle datasets download -d zynicide/wine-reviews # Use this to get data if you want it.

In [3]:
df = pd.read_csv('./data/train.csv', engine='python')

In [4]:
def clean_dataframe(df):
    # Transformations here.
    new_df = df.copy() 
    new_df = new_df.drop(columns=['Name', 'Ticket', 'Cabin'])

    new_df['Age'] = new_df['Age'].fillna(new_df.Age.median())
    
    new_df = pd.get_dummies(new_df, columns=['Sex', 'Embarked', 'Pclass'])
    new_df = pd.DataFrame(minmax_scale(new_df), columns=new_df.columns)

    return new_df

def test_data(test_df, ans_df):
    # For local testing so I don't have to submit all the time.
    correct = 0
    total = 0 
    for index, row in test_df.iterrows():
        if row['PassengerId'] in ans_df['PassengerId'].values:
            if ans_df.loc()[ans_df['PassengerId'] == row['PassengerId']].iloc[0]['Survived'] == row['Survived']:
                correct = correct +1
        total += 1 
    return "{} out of {} for a % of {}".format(correct, total, correct/total)

In [5]:
clean_data = clean_dataframe(df)
labels = clean_data['Survived'].copy()
corr_matrix = clean_data.corr()
clean_data = clean_data.drop(columns=['Survived', 'PassengerId'])
X_train, X_test, y_train, y_test = train_test_split(clean_data, labels, random_state=42)

In [6]:
corr_matrix['Survived'].sort_values(ascending=False)

Survived       1.000000
Sex_female     0.543351
Pclass_1       0.285904
Fare           0.257307
Embarked_C     0.168240
Pclass_2       0.093349
Parch          0.081629
Embarked_Q     0.003650
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.064910
Embarked_S    -0.155660
Pclass_3      -0.322308
Sex_male      -0.543351
Name: Survived, dtype: float64

In [9]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
model = gbc
gbc.predict([clean_data.iloc[1]])[0] 

1.0

In [7]:
gbc = GradientBoostingClassifier()
# hyper parameter optimization
param_grid = {'max_depth': [3, 4, 6, 8, 10, None],
 'learning_rate': [.01, .1, 1, 10],
 'n_estimators': [200, 400, 600, 800, 1000]}

gbc_search = RandomizedSearchCV(gbc, cv=5, param_distributions=param_grid, scoring='neg_mean_squared_error',
                            n_iter=25, verbose=2, n_jobs=-1)
gbc_search.fit(X_train, y_train)
gbc_search.predict([clean_data.iloc[1]])[0]
hpo_model = gbc_search
# Mmmm, MSE is pretty low, but idk. might be the best?
# 82.5 test, and the best MSE as well.

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  1.2min finished


In [19]:
forest_reg = RandomForestClassifier()

# hyper parameter optimization
param_grid = {'bootstrap': [True, False],
 'max_depth': [3, 4, 6, 8, 10, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2],
 'min_samples_split': [2, 5],
 'n_estimators': [200, 400, 600, 800, 1000]}

r_search = RandomizedSearchCV(forest_reg, cv=5, param_distributions=param_grid, scoring='neg_mean_squared_error',
                            n_iter=25, verbose=2, n_jobs=-1)
r_search.fit(X_train, y_train)
r_search.predict([clean_data.iloc[1]])[0]
forest_model = r_search

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:   24.7s finished


In [ ]:
knn = KNeighborsClassifier(n_neighbors=2) # can play with this a bit.
knn.fit(X_train,y_train)
model = knn
knn.predict([clean_data.iloc[1]])[0] # sanity check

In [26]:
comp_df = pd.read_csv('./data/test.csv', engine='python')
comp_df_passengerId = comp_df['PassengerId'].copy()
comp_clean_df = clean_dataframe(comp_df)
comp_clean_df['Fare'] = comp_clean_df['Fare'].fillna(comp_clean_df.Fare.median())
comp_clean_df = comp_clean_df.drop(columns=['PassengerId'])
comp_clean_df['Survived'] = comp_clean_df.apply(lambda row: model.predict([row])[0], axis=1)
#comp_clean_df['Survived'] = comp_clean_df.apply(lambda row: hpo_model.predict([row])[0], axis=1)
#comp_clean_df['Survived'] = comp_clean_df.apply(lambda row: forest_model.predict([row])[0], axis=1)
final_df = pd.DataFrame(list(zip(comp_df_passengerId, comp_clean_df['Survived'])),columns=['PassengerId', 'Survived'])

In [27]:
final_df.to_csv('output_gbc.csv', index=False)
#!kaggle competitions submit -c titanic -f ./output.csv

In [21]:
# To see if it should be better before submitting.
ans_df = pd.read_csv('./data/answers.csv', engine='python')
ans_df = ans_df.rename({1: 'PassengerId', 2:'Survived'}, axis='columns')

test_df = comp_df.merge(ans_df, left_on="Name", right_on="name")
test_df_only_survived = test_df[['survived', 'PassengerId']].copy()
test_df_only = test_df_only_survived.rename({'survived': 'Survived'}, axis='columns')
ans_df = test_df_only
ans_df = ans_df.drop(1) # Something weird happens and we have to drop this one.
print(test_data(final_df, ans_df))
#ans_df = ans_df[ans_df['name'] == tr_og_df['Name']]

#ans_df.drop(ans_df.index[0:892], inplace=True) # We can't do this as test is not sorted.
#final_df = final_df.drop([417], axis="index") # We have to do this before testing for some reason.

308 out of 418 for a % of 0.7368421052631579


In [25]:
print("{}, {}".format(model.score(X_train, y_train), model.score(X_test, y_test))) 
# even our training data model can't get a ton right.
# C is huge, so what does this mean? it means we should be overfitting, but we're not. Do we need a 
# more complex model?

0.9086826347305389, 0.8251121076233184


In [ ]:
mean_squared_error(model.predict(X_test), y_test)